# ACHAL GANDHI
# 218319111
# Final math 4171

### **Question (1)**
## **Implementations and Results of Nelder-Meader, Steepdest Descent, DFP, and BFGS**
## **Table is created at end of question 1**

In [5]:
import math
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import scipy.optimize as opt
import matplotlib.pyplot as plt
from prettytable import PrettyTable
import warnings
import time

# **Defining powell_bs, brown_den, gulf function**

In [6]:
def powell_bs(par):
  par=np.append(0,par)
  s=len(par)
  h=np.zeros((s,s))
  x = par[1]
  y = par[2]

  f1 = 1e4 * x * y - 1
  f2 = np.exp(-x) + np.exp(-y) - 1.0001

  a = 2e4 * f1
  b = 2 * f2


  fn = f1 * f1 + f2 * f2
  gr = np.array([y * a - np.exp(-x) * b, x * a - np.exp(-y) * b])

  x1 = par[1]
  x2 = par[2]

  t1 = 1.0e+04*x1*x2 - 1
  t2 = np.exp( - x1 ) + np.exp( - x2 ) - 1.0001
  h[1,1] = 2.0*( 1.0e+8*x2 ** 2 + t2*np.exp( - x1 ) + np.exp( - x1 ) ** 2 )
  h[1,2] = 2.0*( t1*1.0e+04 + 1.0e+8*x1*x2 + np.exp( - x1 - x2 ) )
  h[2,2] = 2.0*( 1.0e+8*x1 ** 2 + t2*np.exp( - x2 ) + np.exp( - x2 ) ** 2 )
  h[2,1] = h[1,2]

  h=np.delete(h,0,0)
  h=np.delete(h,0,1)
  return(np.array([fn,gr,h],dtype=object))

x0 = np.array([1, 1])
def brown_den(par):
  par=np.append(0,par)
  s=len(par)
  h=np.zeros((s,s))
  m=20
  x1 = par[1]
  x2 = par[2]
  x3 = par[3]
  x4 = par[4]

  ti = np.linspace(1,m,num=m) * 0.2
  sinti = np.sin(ti)
  l = x1 + ti * x2 - np.exp(ti)
  r = x3 + x4 * sinti - np.cos(ti)
  f = l * l + r * r
  lf4 = 4 * l * f
  rf4 = 4 * r * f

  fsum = sum(f * f)
  grad = np.array([sum(lf4),sum(lf4 * ti),sum(rf4),sum(rf4 * sinti)])


  for i in range(1,m+1):
    d1 = i/5.0
    d2 = np.sin( d1 )
    t1 = x1 + d1*x2 - np.exp( d1 )
    t2 = x3 + d2*x4 - np.cos( d1 )
    t = 8.0 * t1 * t2
    s1 = 12.0*t1 ** 2 + 4.0*t2 ** 2
    s2 = 12.0*t2 ** 2 + 4.0*t1 ** 2
    h[1,1] = h[1,1] + s1
    h[1,2] = h[1,2] + s1*d1
    h[2,2] = h[2,2] + s1*d1 ** 2
    h[1,3] = h[1,3] + t
    h[2,3] = h[2,3] + t*d1
    h[3,3] = h[3,3] + s2
    h[1,4] = h[1,4] + t*d2
    h[2,4] = h[2,4] + t*d1*d2
    h[3,4] = h[3,4] + s2*d2
    h[4,4] = h[4,4] + s2*d2 ** 2

  h[2,1] = h[1,2]
  h[3,1] = h[1,3]
  h[3,2] = h[2,3]
  h[4,1] = h[1,4]
  h[4,2] = h[2,4]
  h[4,3] = h[3,4]


  h=np.delete(h,0,0)
  h=np.delete(h,0,1)
  fn=fsum
  gr=grad
  return(np.array([fn,gr,h],dtype=object))

a1 = np.array([25, 5, -5, 1])
def gulf(par):
  m=90
  par=np.append(0,par)
  s=len(par)
  h=np.zeros((s,s))
  p66 = 2 / 3

  x1 = par[1]
  x2 = par[2]
  x3 = par[3]

  ti = np.linspace(1,m,num=m) * 0.01
  y = 25 + (-50 * np.log(ti)) ** p66

  x2y = x2 - y
  ax2y = abs(x2y)
  x2yz = ax2y ** x3
  e = np.exp(-x2yz / x1)
  fi = e - ti
  efxyz2 = 2 * e * fi * x2yz

  dx = sum(efxyz2 / (x1 * x1))
  dy = -sum(efxyz2 * x3 / (x1 * x2y))
  dz = -sum(efxyz2 * np.log(ax2y) / x1)

  fsum = sum(fi * fi)
  grad = np.array([dx, dy, dz])

  d1 = p66
  for i in range(1,m+1):
      arg = 0.01*i
      r = ( -50.0*np.log( arg ) ) ** d1 + 25.0 - x2
      t1 = abs( r ) ** x3/x1
      t2 = np.exp( -t1 )
      t3 = t1*t2*( t1*t2 + ( t1 - 1.0 )*( t2 - arg ) )
      t = t1*t2*( t2 - arg )
      logr = np.log( abs( r ) )
      h[1,1] = h[1,1] + t3 - t
      h[1,2] = h[1,2] + t3/r
      h[2,2] = h[2,2] + ( t + x3*t3 )/r ** 2
      h[1,3] = h[1,3] - t3*logr
      h[2,3] = h[2,3] + (t-x3*t3*logr)/r
      h[3,3] = h[3,3] + t3*logr ** 2

  h[1,1] = h[1,1] / x1 ** 2
  h[1,2] = h[1,2]*x3/x1
  h[2,2] = h[2,2]*x3
  h[1,3] = h[1,3] / x1
  h = 2.0 * h
  h[2,1] = h[1,2]
  h[3,1] = h[1,3]
  h[3,2] = h[2,3]

  h=np.delete(h,0,0)
  h=np.delete(h,0,1)
  fn=fsum
  gr=grad
  return(np.array([fn,gr,h],dtype=object))

a2 = np.array([5, 2.5, 0.15])

## **Wrapper functions for separating function, gradient, hessian**

In [7]:
def pbo(par):
  result = powell_bs(par)
  return result[0]

def bdo(par):
  result = brown_den(par)
  return result[0]

def gulfo(par):
  result = gulf(par)
  return result[0]

def pbg(par):
  result = powell_bs(par)
  return result[1]

def bdg(par):
  result = brown_den(par)
  return result[1]

def gulfg(par):
  result = gulf(par)
  return result[1]

def pbh(par):
  result = powell_bs(par)
  return result[2]

def bdh(par):
  result = brown_den(par)
  return result[2]

def gulfh(par):
  result = gulf(par)
  return result[2]



### **Nelder Mead Algorithm implementation and results.**

In [8]:
def nelder_mead(obj_func, initial_guess, max_iter=1000, tol=1e-6, alpha=1, beta=0.5, gamma=2, delta=0.5):
    """
    Perform optimization using the Nelder-Mead algorithm.

    Parameters:
    - obj_func: Objective function to be minimized.
    - initial_guess: Starting point for the optimization.
    - max_iter: Maximum number of iterations.
    - tol: Tolerance for convergence.
    - alpha: Reflection coefficient.
    - beta: Contraction coefficient.
    - gamma: Expansion coefficient.
    - delta: Shrinkage coefficient.

    Returns:
    - The best point found.
    - Number of iterations performed.
    - Elapsed time.
    - Boolean indicating if optimality was reached.
    """
    n = len(initial_guess)
    # Initialize a simplex around the initial guess
    vertices = np.zeros((n + 1, n))
    vertices[0] = initial_guess

    # Create initial simplex
    for i in range(n):
        vertex = initial_guess.copy()
        vertex[i] = initial_guess[i] + 1.0
        vertices[i + 1] = vertex

    start_time = time.time()

    for iteration in range(max_iter):
        # Evaluate the objective function for all vertices
        values = np.array([obj_func(vertex) for vertex in vertices])

        # Sort vertices by their function values
        sorted_indices = np.argsort(values)
        vertices = vertices[sorted_indices]

        # Calculate centroid of all points except the worst
        centroid = np.mean(vertices[:-1], axis=0)

        # Reflection step
        reflected_vertex = centroid + alpha * (centroid - vertices[-1])
        reflected_value = obj_func(reflected_vertex)

        if values[0] <= reflected_value < values[-2]:
            # If the reflected point is better than the second worst, replace the worst point
            vertices[-1] = reflected_vertex
        elif reflected_value < values[0]:
            # Expansion step if the reflected point is the best so far
            expanded_vertex = centroid + gamma * (reflected_vertex - centroid)
            expanded_value = obj_func(expanded_vertex)

            # Choose between reflected and expanded point
            if expanded_value < reflected_value:
                vertices[-1] = expanded_vertex
            else:
                vertices[-1] = reflected_vertex
        else:
            # Contraction step
            contracted_vertex = centroid + beta * (vertices[-1] - centroid)
            contracted_value = obj_func(contracted_vertex)

            if contracted_value < values[-1]:
                vertices[-1] = contracted_vertex
            else:
                # Shrink the simplex towards the best point
                for i in range(1, n + 1):
                    vertices[i] = vertices[0] + delta * (vertices[i] - vertices[0])

        # Check for convergence
        if np.linalg.norm(vertices[0] - vertices[-1]) < tol:
            end_time = time.time()
            elapsed_time = end_time - start_time
            return vertices[0], iteration + 1, elapsed_time, True

    end_time = time.time()
    elapsed_time = end_time - start_time

    # Return the best point found, number of iterations, running time, and whether optimality was reached
    return vertices[0], max_iter, elapsed_time, False

# Suppress runtime warnings that may occur during optimization
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)
    result_powell_bs, iter_powell_bs, time_powell_bs, opt_powell_bs = nelder_mead(pbo, x0)
    result_brown_den, iter_brown_den, time_brown_den, opt_brown_den = nelder_mead(bdo, a1)
    result_gulf, iter_gulf, time_gulf, opt_gulf = nelder_mead(gulfo, a2)

# Print results for the Nelder-Mead algorithm
print("Results for Nelder-Mead algorithm")
print("Result for powell_bs:", result_powell_bs)
print("Number of iterations for powell_bs:", iter_powell_bs)
print("Running time for powell_bs:", time_powell_bs)
print("Optimality reached for powell_bs:", opt_powell_bs)

print("\nResult for brown_den:", result_brown_den)
print("Number of iterations for brown_den:", iter_brown_den)
print("Running time for brown_den:", time_brown_den)
print("Optimality reached for brown_den:", opt_brown_den)

print("\nResult for gulf:", result_gulf)
print("Number of iterations for gulf:", iter_gulf)
print("Running time for gulf:", time_gulf)
print("Optimality reached for gulf:", opt_gulf)


Results for Nelder-Mead algorithm
Result for powell_bs: [9.10614509e+00 1.09815953e-05]
Number of iterations for powell_bs: 509
Running time for powell_bs: 1.2984683513641357
Optimality reached for powell_bs: True

Result for brown_den: [-11.59443956  13.20363     -0.40343998   0.23677878]
Number of iterations for brown_den: 214
Running time for brown_den: 1.726170539855957
Optimality reached for brown_den: True

Result for gulf: [50.00000249 24.99999994  1.50000002]
Number of iterations for gulf: 274
Running time for gulf: 7.42261815071106
Optimality reached for gulf: True


## **Steepdest Descent Algorithm implementation and results.**

In [9]:
def cubicmin(a, fa, fpa, b, fb, c, fc):
    """
    Uses cubic interpolation to estimate the minimum of a function based on function and gradient
    values at three points.
    """
    with np.errstate(divide='raise', over='raise', invalid='raise'):
        try:
            # Coefficients for cubic interpolation
            C = fpa
            db = b - a
            dc = c - a
            denom = (db * dc) ** 2 * (db - dc)
            d1 = np.empty((2, 2))
            d1[0, 0] = dc ** 2
            d1[0, 1] = -db ** 2
            d1[1, 0] = -dc ** 3
            d1[1, 1] = db ** 3
            # Linear system to solve for cubic coefficients A and B
            [A, B] = np.dot(d1, np.asarray([fb - fa - C * db, fc - fa - C * dc]).flatten())
            A /= denom
            B /= denom
            radical = B * B - 3 * A * C

            # Discriminant must be non-negative; otherwise, take it as zero
            radical = max(radical, 0)

            # Minimizer for the cubic polynomial
            xmin = a + (-B + np.sqrt(radical)) / (3 * A)
        except ArithmeticError:
            # Handle any mathematical errors
            return None
    # Ensure the result is a finite number
    if not np.isfinite(xmin):
        return None
    return xmin

# Function to find the minimum of a quadratic polynomial
def quadmin(a, fa, fpa, b, fb):
    """
    Uses quadratic interpolation to estimate the minimum of a function based on function
    values at two points and the gradient at one point.
    """
    try:
        D, C, db = fa, fpa, b - a
        B = (fb - D - C * db) / (db * db)
        xmin = a - C / (2.0 * B)
    except ArithmeticError:
        # Handle any mathematical errors
        return None
    # Ensure the result is a finite number
    return xmin if np.isfinite(xmin) else None

# Helper function used in the line search to refine the step size
def zoom(a_lo, a_hi, phi_lo, phi_hi, derphi_lo, phi, derphi, phi0, derphi0, c1, c2):
    """
    Part of the line search algorithm that zooms in on an interval where the Wolfe conditions
    are likely to be satisfied.
    """
    maxiter, i, delta1, delta2, phi_rec, a_rec = 100, 0, 0.2, 0.1, phi0, 0

    while True:
        dalpha = a_hi - a_lo
        a, b = (a_hi, a_lo) if dalpha < 0 else (a_lo, a_hi)

        # Use cubic interpolation if we have a previous value (phi_rec)
        if (i > 0):
            cchk = delta1 * dalpha
            a_j = cubicmin(a_lo, phi_lo, derphi_lo, a_hi, phi_hi, a_rec, phi_rec)
        # Use quadratic interpolation otherwise
        if (i == 0) or (a_j is None) or (a_j > b - cchk) or (a_j < a + cchk):
            qchk = delta2 * dalpha
            a_j = quadmin(a_lo, phi_lo, derphi_lo, a_hi, phi_hi)
            if (a_j is None) or (a_j > b - qchk) or (a_j < a + qchk):
                a_j = a_lo + 0.5 * dalpha

        # Evaluate the function and its derivative at the new step size a_j
        phi_aj = phi(a_j)
        if (phi_aj > phi0 + c1 * a_j * derphi0) or (phi_aj >= phi_lo):
            phi_rec, a_rec, a_hi, phi_hi = phi_hi, a_hi, a_j, phi_aj
        else:
            derphi_aj = derphi(a_j)
            if abs(derphi_aj) <= -c2 * derphi0:
                # Wolfe conditions are satisfied
                return a_j, phi_aj, derphi_aj
            if derphi_aj * (a_hi - a_lo) >= 0:
                phi_rec, a_rec, a_hi, phi_hi = phi_hi, a_hi, a_lo, phi_lo
            else:
                phi_rec, a_rec = phi_lo, a_lo
            a_lo, phi_lo, derphi_lo = a_j, phi_aj, derphi_aj
        i += 1
        if i > maxiter:
            print('Failed to find a conforming step size')
            return None, None, None

# Wolfe conditions line search
def line_search_wolfe(fn, fnp, xk, pk, c1 = 1e-4, c2 = 0.9, amax = np.Inf, maxiter = 10):
    """
    Performs a line search to satisfy the Wolfe conditions.
    """
    func_it = [0]
    grad_it = [0]
    grad_0 = [None]
    grad_a = [None]

    def phi(alpha):
        # Function value for line search
        func_it[0] += 1
        return fn(xk + alpha*pk)

    def phip(alpha):
        # Derivative value for line search
        grad_it[0] += 1
        grad_0[0] = fnp(xk + alpha*pk)
        grad_a[0] = alpha
        return np.dot(grad_0[0], pk)

    # Evaluate the function and its gradient at the initial point
    phi_0 = phi(0)
    phip_0 = phip(0)

    # Initial guess for the step size
    a0 = 0
    a1 = 1.0

    # Ensure that the step size does not exceed the maximum allowed value
    if amax is not None:
        a1 = min(a1, amax)

    # Main loop for the line search
    for i in range(maxiter):
        # Evaluate the function at the new step size
        phi_a1 = phi(a1)
        phi_a0 = phi_0
        phip_a0 = phip_0

        not_first_iteration = i > 0
        # Check if the Wolfe conditions are satisfied or need to zoom in
        if (phi_a1 > phi_0 + c1 * a1 * phip_0) or \
               ((phi_a1 >= phi_a0) and not_first_iteration):
            a_star, phi_star, phip_star = zoom(a0, a1, phi_a0,
                                  phi_a1, phip_a0, phi, phip,
                                  phi_0, phip_0, c1, c2)
            break

        phip_a1 = phip(a1)
        if (abs(phip_a1) <= -c2*phip_0):
          a_star = a1
          phi_star = phi_a1
          phip_star = phip_a1
          break

        if(phip_a1 >= 0):
          a_star, phi_star, phip_star = zoom(a0, a1, phi_a0,
                                  phi_a1, phip_a0, phi, phip,
                                  phi_0, phip_0, c1, c2)
          break

        # Update the step size for the next iteration
        a2 = 2 * a1
        if amax is not None:
          a2 = min(a2, amax)
        a0 = a1
        a1 = a2
        phi_a0 = phi_a1
        phi_a1 = phi(a1)
        phip_a0 = phip_a1
    else:
        # If the maximum number of iterations is reached without satisfying the conditions
        a_star = a1
        phi_star = phi_a1
        phip_star = None
        print('The line search algorithm did not converge')

    if phip_star is None:
        print('The line search algorithm did not converge')
    else:
        phip_star = grad_0[0]

    return a_star

# Steepest descent optimization algorithm
def steepest_descent(fn, fnp, xk, maxiter=100000):
    """
    Minimizes a function using the steepest descent method.
    """
    k = 0
    start_time = time.time()

    while k < maxiter:
        gk = fnp(xk)  # Calculate the gradient at the current point

        if np.linalg.norm(gk) <= 1e-3:
            break  # Check for optimality

        alpha = line_search_wolfe(fn, fnp, xk, -gk)  # Perform line search to find the step size

        if alpha is None:
            # Handle cases where the line search fails to find a step size
            print("Failed to find a conforming step size. Using a small default step size.")
            alpha = 1e-4

        xk = xk + alpha * (-gk)  # Update the current point using the step size
        k += 1  # Increment iteration counter

    end_time = time.time()
    elapsed_time = end_time - start_time  # Calculate the elapsed time

    optimality_reached = np.linalg.norm(fnp(xk)) <= 1e-3  # Check if the solution is optimal

    return xk, k, elapsed_time, optimality_reached

# Suppress runtime warnings related to numerical issues during optimization
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)
    result_powell_bs1, iter_powell_bs1, time_powell_bs1, opt_powell_bs1 = steepest_descent(pbo, pbg, x0)
    result_brown_den1, iter_brown_den1, time_brown_den1, opt_brown_den1 = steepest_descent(bdo, bdg, a1)
    result_gulf1, iter_gulf1, time_gulf1, opt_gulf1 = steepest_descent(gulfo, gulfg, a2)

# Output the results for the steepest descent algorithm
print("Results for Steepest Descent algorithm\n")
print("Result for powell_bs:", result_powell_bs1)
print("Number of iterations for powell_bs:", iter_powell_bs1)
print("Running time for powell_bs:", time_powell_bs1)
print("Optimality reached for powell_bs:", opt_powell_bs1)

print("\nResult for brown_den:", result_brown_den1)
print("Number of iterations for brown_den:", iter_brown_den1)
print("Running time for brown_den:", time_brown_den1)
print("Optimality reached for brown_den:", opt_brown_den1)

print("\nResult for gulf:", result_gulf1)
print("Number of iterations for gulf:", iter_gulf1)
print("Running time for gulf:", time_gulf1)
print("Optimality reached for gulf:", opt_gulf1)


Results for Steepest Descent algorithm

Result for powell_bs: [0.01004816 0.01004816]
Number of iterations for powell_bs: 12
Running time for powell_bs: 0.06866860389709473
Optimality reached for powell_bs: True

Result for brown_den: [-11.59443991  13.20363006  -0.40343971   0.23677947]
Number of iterations for brown_den: 450
Running time for brown_den: 5.332188367843628
Optimality reached for brown_den: True

Result for gulf: [35.02836788 25.67770834  1.39301742]
Number of iterations for gulf: 3059
Running time for gulf: 35.49288463592529
Optimality reached for gulf: True


## **DFP Algorithm implementation and results.**

In [10]:
def minimizing_function(f, grad_f, xk, dk, alpha_init=1, c1=1e-4, c2=0.9, max_iter=100):
    """
    Parameters:
    - f: Objective function to minimize.
    - grad_f: Gradient of the objective function.
    - xk: Current point in the parameter space.
    - dk: Current search direction.
    - alpha_init: Initial guess for the step size.
    - c1: Constant for the Wolfe condition related to sufficient decrease.
    - c2: Constant for the Wolfe condition related to the curvature.
    - max_iter: Maximum iterations for line search.

    Returns:
    - alpha: Step size that satisfies Wolfe conditions.
    - i + 1: Number of iterations taken to find the step size.
    """
    alpha = alpha_init
    for i in range(max_iter):
        # First Wolfe condition: Armijo rule for sufficient decrease
        if f(xk + alpha * dk) > f(xk) + c1 * alpha * np.dot(grad_f(xk), dk):
            alpha *= 0.5  # If not satisfied, reduce alpha
        # Second Wolfe condition: Curvature condition
        elif np.dot(grad_f(xk + alpha * dk), dk) < c2 * np.dot(grad_f(xk), dk):
            alpha *= 1.1  # If not satisfied, increase alpha
        else:
            # If both conditions are satisfied, return the step size
            return alpha, i + 1
    # If Wolfe conditions are not satisfied within max_iter, raise an error
    raise ValueError("Wolfe line search did not converge within the maximum number of iterations")

# Updated DFP method with Wolfe line search
def minimize_dfp(f, grad_f, x0, max_iter=100, epsilon=1e-5):
    """
    Minimize a function using the Davidon-Fletcher-Powell (DFP) algorithm.

    Parameters:
    - f: Objective function to minimize.
    - grad_f: Gradient of the objective function.
    - x0: Initial guess for the parameters.
    - max_iter: Maximum iterations for the DFP algorithm.
    - epsilon: Tolerance for the stopping criterion based on the gradient norm.

    Returns:
    - xk: Estimated position of the minimum.
    - iteration_count: Number of iterations taken to converge.
    - running_time: Time taken to converge.
    - optimality_reached: Boolean indicating if optimality was reached.
    """
    n = len(x0)
    xk = x0
    Hk = np.eye(n)  # Initialize Hessian approximation to the identity matrix
    iteration_count = 0  # Initialize iteration counter
    optimality_reached = False  # Initialize optimality flag

    start_time = time.time()  # Start timer

    for k in range(max_iter):
        iteration_count += 1
        gk = grad_f(xk)
        if np.linalg.norm(gk) < epsilon:
            optimality_reached = True  # Check if optimality condition is met
            break

        dk = -np.dot(Hk, gk)  # Calculate search direction
        try:
            step_size, iters = minimizing_function(f, grad_f, xk, dk)  # Wolfe line search
        except ValueError as e:
            print(str(e))
            break

        xk_new = xk + step_size * dk  # Update position
        gk_new = grad_f(xk_new)
        yk = gk_new - gk
        sk = xk_new - xk
        rho_k = 1.0 / np.dot(yk, sk)
        # Update Hessian approximation using DFP formula
        Hk_new = (np.eye(n) - rho_k * np.outer(sk, yk)) @ Hk @ (np.eye(n) - rho_k * np.outer(yk, sk)) + rho_k * np.outer(sk, sk)
        xk = xk_new
        Hk = Hk_new

    end_time = time.time()  # End timer
    running_time = end_time - start_time  # Calculate elapsed time

    return xk, iteration_count, running_time, optimality_reached

# Suppress warnings that may occur during the optimization process
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)
    result_powell_bs3 ,iter_powell_bs3, time_powell_bs3, opt_powell_bs3 = minimize_dfp(pbo, pbg, x0)
    result_brown_den3, iter_brown_den3, time_brown_den3, opt_brown_den3 = minimize_dfp(bdo, bdg, a1)
    result_gulf3, iter_gulf3, time_gulf3, opt_gulf3 = minimize_dfp(gulfo, gulfg, a2)

# Print results for the DFP algorithm
print("Results for DFP algorithm\n")
print("Result for powell_bs:", result_powell_bs3)
print("Number of iterations for powell_bs:", iter_powell_bs3)
print("Running time for powell_bs:", time_powell_bs3)
print("Optimality reached for powell_bs:", opt_powell_bs3)

print("\nResult for brown_den:", result_brown_den3)
print("Number of iterations for brown_den:", iter_brown_den3)
print("Running time for brown_den:", time_brown_den3)
print("Optimality reached for brown_den:", opt_brown_den3)

print("\nResult for gulf:", result_gulf3)
print("Number of iterations for gulf:", iter_gulf3)
print("Running time for gulf:", time_gulf3)
print("Optimality reached for gulf:", opt_gulf3)



Results for DFP algorithm

Result for powell_bs: [-0.00994809 -0.00994809]
Number of iterations for powell_bs: 54
Running time for powell_bs: 0.04343056678771973
Optimality reached for powell_bs: True

Result for brown_den: [-11.5944399   13.20363005  -0.40343949   0.23677877]
Number of iterations for brown_den: 26
Running time for brown_den: 0.11340570449829102
Optimality reached for brown_den: True

Result for gulf: [50.00004358 24.99998541  1.49999971]
Number of iterations for gulf: 31
Running time for gulf: 0.3435060977935791
Optimality reached for gulf: True


## **BFGS implementation and results**

In [11]:
def minimize_bfgs(fn, fnp, xk, maxiter=100000):
    """
    Minimize a function using the BFGS algorithm.

    :param fn: The function to minimize.
    :param fnp: The gradient of the function.
    :param xk: Initial guess for the variables.
    :param maxiter: Maximum number of iterations allowed.
    :return: Tuple containing the optimal solution, number of iterations, elapsed time, and optimality flag.
    """
    k = 0
    start_time = time.time()

    # Initialize the inverse Hessian approximation as the identity matrix
    n = len(xk)
    Bk = np.eye(n)

    while k < maxiter:
        # Calculate the gradient at the current point
        gk = fnp(xk)

        # Termination condition: if the norm of the gradient is small enough
        if np.linalg.norm(gk) <= 1e-3:
            break

        # Compute the search direction using the BFGS update
        pk = -np.dot(Bk, gk)

        # Perform a line search to find an acceptable step size that satisfies Wolfe conditions
        alpha = line_search_wolfe(fn, fnp, xk, pk)

        # If line search fails, use a default small step size
        if alpha is None:
            print("Failed to find a conforming step size. Using a small default step size.")
            alpha = 1e-4

        # Update the position
        xk_new = xk + alpha * pk

        # Compute sk and yk vectors for BFGS update
        sk = xk_new - xk
        yk = fnp(xk_new) - gk
        rho_k = 1.0 / np.dot(yk, sk)
        A1 = np.eye(n) - rho_k * np.outer(sk, yk)
        A2 = np.eye(n) - rho_k * np.outer(yk, sk)
        Bk = np.dot(A1, np.dot(Bk, A2)) + rho_k * np.outer(sk, sk)

        # Update the current estimate
        xk = xk_new
        k += 1

    end_time = time.time()
    elapsed_time = end_time - start_time

    # Check if the solution is optimal
    optimality_reached = np.linalg.norm(fnp(xk)) <= 1e-3

    return xk, k, elapsed_time, optimality_reached

# Ignore runtime warnings that may occur during optimization
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)
    # Call minimize_bfgs function with the problem-specific functions and initial guesses
    result_powell_bs2, iter_powell_bs2, time_powell_bs2, opt_powell_bs2 = minimize_bfgs(pbo, pbg, x0)
    result_brown_den2, iter_brown_den2, time_brown_den2, opt_brown_den2 = minimize_bfgs(bdo, bdg, a1)
    result_gulf2, iter_gulf2, time_gulf2, opt_gulf2 = minimize_bfgs(gulfo, gulfg, a2)

# Output the results
print("Results for BFGS algorithm\n")
print("Result for powell_bs:", result_powell_bs2)
print("Number of iterations for powell_bs:", iter_powell_bs2)
print("Running time for powell_bs:", time_powell_bs2)
print("Optimality reached for powell_bs:", opt_powell_bs2)

print("\nResult for brown_den:", result_brown_den2)
print("Number of iterations for brown_den:", iter_brown_den2)
print("Running time for brown_den:", time_brown_den2)
print("Optimality reached for brown_den:", opt_brown_den2)

print("\nResult for gulf:", result_gulf2)
print("Number of iterations for gulf:", iter_gulf2)
print("Running time for gulf:", time_gulf2)
print("Optimality reached for gulf:", opt_gulf2)


Results for BFGS algorithm

Result for powell_bs: [-0.00994847 -0.0099477 ]
Number of iterations for powell_bs: 51
Running time for powell_bs: 0.03192448616027832
Optimality reached for powell_bs: True

Result for brown_den: [-11.59443988  13.20363005  -0.4034395    0.23677872]
Number of iterations for brown_den: 19
Running time for brown_den: 0.06423521041870117
Optimality reached for brown_den: True

Result for gulf: [52.5403872  24.90590268  1.51471642]
Number of iterations for gulf: 15
Running time for gulf: 0.13026642799377441
Optimality reached for gulf: True


## **Here is table with results**

In [12]:
table = PrettyTable()

table.field_names = ["Method", "Powell_BS Result", "Powell_BS Iterations", "Powell_BS Time", "Powell_BS Optimality",
                     "Brown_Den Result", "Brown_Dennis Iterations", "Brown_Den Time", "Brown_Den Optimality",
                     "Gulf Result", "Gulf Iterations", "Gulf Time", "Gulf Optimality"]

table.add_row(["Nelder-Mead", result_powell_bs, iter_powell_bs, time_powell_bs, opt_powell_bs,
               result_brown_den, iter_brown_den, time_brown_den, opt_brown_den,
               result_gulf, iter_gulf, time_gulf, opt_gulf])

table.add_row(["Steepest Descent", result_powell_bs1, iter_powell_bs1, time_powell_bs1, opt_powell_bs1,
               result_brown_den1, iter_brown_den1, time_brown_den1, opt_brown_den1,
               result_gulf1, iter_gulf1, time_gulf1, opt_gulf1])

table.add_row(["DFP", result_powell_bs3, iter_powell_bs3, time_powell_bs3, opt_powell_bs3,
               result_brown_den3, iter_brown_den3, time_brown_den3, opt_brown_den3,
               result_gulf3, iter_gulf3, time_gulf3, opt_gulf3])

table.add_row(["BFGS", result_powell_bs2, iter_powell_bs2, time_powell_bs2, opt_powell_bs2,
               result_brown_den2, iter_brown_den2, time_brown_den2, opt_brown_den2,
               result_gulf2, iter_gulf2, time_gulf2, opt_gulf2])

print(table)


+------------------+---------------------------------+----------------------+---------------------+----------------------+-------------------------------------------------------+-------------------------+---------------------+----------------------+---------------------------------------+-----------------+---------------------+-----------------+
|      Method      |         Powell_BS Result        | Powell_BS Iterations |    Powell_BS Time   | Powell_BS Optimality |                    Brown_Den Result                   | Brown_Dennis Iterations |    Brown_Den Time   | Brown_Den Optimality |              Gulf Result              | Gulf Iterations |      Gulf Time      | Gulf Optimality |
+------------------+---------------------------------+----------------------+---------------------+----------------------+-------------------------------------------------------+-------------------------+---------------------+----------------------+---------------------------------------+---------------

### **Question 2**

In [13]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

# Load MNIST data using sklearn
mnist = fetch_openml('mnist_784', version=1)

# Preprocess the data
# Select the first 100 images and labels from the dataset
X = mnist.data.iloc[:100].values / 255.0  # Normalize
y = mnist.target.iloc[:100].values.astype(int)

# One-hot encode the labels
y_one_hot = np.zeros((y.size, y.max()+1))
y_one_hot[np.arange(y.size), y] = 1

# Print shapes to confirm
X.shape, y_one_hot.shape

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


((100, 784), (100, 10))

# **Defining neural network with its functions and using steepest descent to minimize entropy loss**

# **Changes have been made to steepest descent algorithm for neural network**

In [14]:
import time
# Activation functions and their derivatives
def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return (x > 0) * 1.0

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=1, keepdims=True)

# Neural Network Class
class FullyConnectedNN:
    def __init__(self):
        # Initialize weights and biases
        self.W1 = np.random.randn(784, 128) * 0.01
        self.b1 = np.zeros((1, 128))
        self.W2 = np.random.randn(128, 10) * 0.01
        self.b2 = np.zeros((1, 10))

    def forward(self, X):
        # Forward pass
        self.Z1 = np.dot(X, self.W1) + self.b1
        self.A1 = relu(self.Z1)
        self.Z2 = np.dot(self.A1, self.W2) + self.b2
        self.A2 = softmax(self.Z2)
        return self.A2

    def compute_loss(self, y_pred, y_true):
        # Compute the cross-entropy loss
        m = y_true.shape[0]
        loss = -np.sum(y_true * np.log(y_pred + 1e-9)) / m
        return loss

    def backward(self, X, y_true):
        # Backward pass
        m = y_true.shape[0]
        dZ2 = self.A2 - y_true
        dW2 = np.dot(self.A1.T, dZ2) / m
        db2 = np.sum(dZ2, axis=0, keepdims=True) / m

        dA1 = np.dot(dZ2, self.W2.T)
        dZ1 = dA1 * relu_derivative(self.Z1)
        dW1 = np.dot(X.T, dZ1) / m
        db1 = np.sum(dZ1, axis=0, keepdims=True) / m

        gradients = {"dW1": dW1, "db1": db1, "dW2": dW2, "db2": db2}
        return gradients
    def flatten_parameters(self):
        return np.concatenate([self.W1.ravel(), self.b1.ravel(), self.W2.ravel(), self.b2.ravel()])

    def set_parameters(self, flat_params):
        w1_end = 784 * 128
        b1_end = w1_end + 128
        w2_end = b1_end + 128 * 10
        self.W1 = flat_params[:w1_end].reshape(784, 128)
        self.b1 = flat_params[w1_end:b1_end].reshape(1, 128)
        self.W2 = flat_params[b1_end:w2_end].reshape(128, 10)
        self.b2 = flat_params[w2_end:].reshape(1, 10)

    def loss_and_grad(self, flat_params, X, y):
        self.set_parameters(flat_params)
        y_pred = self.forward(X)
        loss = self.compute_loss(y_pred, y)
        gradients = self.backward(X, y)
        grad_flat = np.concatenate([gradients['dW1'].ravel(), gradients['db1'].ravel(),
                                    gradients['dW2'].ravel(), gradients['db2'].ravel()])
        return loss, grad_flat

def steepest_descent(nn, X, y, maxiter=100, tol=1e-3):
    flat_params = nn.flatten_parameters()

    def fn(flat_params):
        loss, _ = nn.loss_and_grad(flat_params, X, y)
        return loss

    def fnp(flat_params):
        _, grad = nn.loss_and_grad(flat_params, X, y)
        return grad

    k = 0
    start_time = time.time()

    while k < maxiter:
        gk = fnp(flat_params)
        if np.linalg.norm(gk) <= tol:
            break  # Convergence check

        alpha = line_search_wolfe(fn, fnp, flat_params, -gk)
        if alpha is None:
            print("Failed to find a conforming step size. Using a small default step size.")
            alpha = 1e-4

        flat_params = flat_params + alpha * (-gk)
        nn.set_parameters(flat_params)  # Update neural network parameters
        k += 1

    end_time = time.time()
    elapsed_time = end_time - start_time
    optimality_reached = np.linalg.norm(fnp(flat_params)) <= tol

    return flat_params, k, elapsed_time, optimality_reached

nn = FullyConnectedNN()
steepest_descent(nn, X, y_one_hot)

(array([ 0.02004413, -0.00427742,  0.00362237, ...,  0.13664968,
        -0.23319778,  0.08141984]),
 88,
 3.4250850677490234,
 True)

### **Nelder meader**

## **It is not possible for colab or normal computer environment to calculate nelder mead optimization on neural network.**
## **fully connected network with a hidden layer of size 128 and an output layer of size 10 leads to a significant number of weights and biases to optimize. Nelder-Mead does not scale well with dimensionality due to its need to maintain a simplex with n + 1 vertices in a n-dimensional space.**
